In [88]:
import pandas as pd
import numpy as np 

In [89]:
train_data = pd.read_csv('../data/train.csv') 
train_data

,title,description,Thriller,Classics,Romance,Mystery,Science,Literature,Fantasy,Historical,Fiction
0,It's Not About the Bike: My Journey Back to Life,It is such an all-American story. A lanky kid ...,0,0,0,0,0,0,0,0,0
1,Trilogía Involuntaria #3The House of the Spirits,In one of the most important and beloved Latin...,0,0,0,0,0,1,1,1,1
2,The Girl Who Drank the Moon,"Every year, the people of the Protectorate lea...",0,0,0,0,0,0,0,0,1
3,Give Me Tomorrow: The Korean War's Greatest Un...,An epic story of valor and sacrifice by a lege...,0,0,0,0,0,0,0,0,1
4,The World of Winnie-the-Pooh,"In 1926, ""Winnie-the-Pooh,"" a collection of st...",0,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2948,Wisdom Revolution #2The Oldest Dance,As one of the war heroes searches the oldest l...,0,0,0,0,1,0,1,0,1
2949,Mercy #3Beg For Mercy,"The fight is on in this installment, Mercy is ...",0,0,1,0,0,0,0,0,1
2950,Halo #1Halo,An angel is sent to Earth on a mission.But fal...,0,0,1,0,0,0,1,0,0
2951,Genius #1Evil Genius,Cadel Piggott has a genius IQ and a fascinatio...,0,0,0,1,1,0,1,0,1


In [90]:
test_data = pd.read_csv('../data/test.csv')
test_data

,title,description
0,Asian Saga: Chronological Order #2Tai-Pan,Set in the turbulent days of the founding of H...
1,Prince of Pain #1Prince of Pain I,When Liam pays a madam for a specific fantasy ...
2,Practical Magic #1Practical Magic,Alternate cover for ISBN 9780425190371 (curren...
3,Asian Saga: Chronological Order #5Noble House,This is an alternate cover edition for ISBN13:...
4,El Diablo #1El Diablo,FROM WALL STREET JOURNAL & USA BESTSELLING AUT...
...,...,...
734,Maybe #1.5Maybe Not,"Colleen Hoover, the New York Times bestselling..."
735,The Saga of Darren Shan #11Lord of the Shadows,Book 11 of The Saga Of Darren Shan. Darren's g...
736,The Highly Sensitive Person: How to Thrive Whe...,Do you have a keen imagination and vivid dream...
737,Bad Girls Don't Die #3As Dead As It Gets,It's been three months since Alexis helplessly...


In [91]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
stop_words.add('.')
stop_words.add(',')

In [92]:
import re
word2idx = {}
lemmatizer = WordNetLemmatizer()
pattern = r'[0-9]'
counter = 0
def tokenize(text):
    global counter
    tokens = word_tokenize(text)
    # print(tokens)
    temp = []
    for token in tokens:
        if (token not in stop_words) and len(token)>3:
            txt = re.sub(pattern, '', token)
            # print(txt)
            txt = lemmatizer.lemmatize(txt).lower()
            # if txt not in stop_words:
            temp.append(txt)
            if txt not in word2idx:
                word2idx[txt] = counter
                counter += 1
    return temp

In [93]:
# import nltk

# nltk.download('wordnet')
# nltk.download('wordnet_ic')

In [98]:
x_train = (train_data['title']+train_data['description']).apply(str).to_numpy()
x_test = (test_data['title']+test_data['description']).apply(str).to_numpy()
y_train = train_data.iloc[:, 2:].to_numpy()

x_train = [tokenize(x) for x in x_train]
x_test = [tokenize(x) for x in x_test]

In [99]:
N_train = len(x_train)
N_test = len(x_test)
D = len(word2idx)
classes = 9

In [100]:
x_train_vector = np.zeros((N_train, D))
x_test_vector = np.zeros((N_test, D))

i = 0
for train in x_train:
    for x in train:
        x_train_vector[i, word2idx[x]] += 1
    i += 1

i = 0
for test in x_test:
    for x in test:
        x_test_vector[i, word2idx[x]] += 1
    i += 1


In [101]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_jobs=2, random_state=0)
model.fit(x_train_vector, y_train)

RandomForestClassifier(n_jobs=2, random_state=0)

In [102]:
model.score(x_train_vector, y_train)

0.9989840839823908

In [ ]:
py = model.predict(x_test_vector)
dicti = {}
genres_list = ['Thriller', 'Classics', 'Romance', 'Mystery', 'Science', 'Literature', 'Fantasy', 'Historical', 'Fiction']
for i in range(len(genres_list)):
    dicti[genres_list[i]] = py[:,i]


In [ ]:
submission = pd.DataFrame(dicti)
submission


,Thriller,Classics,Romance,Mystery,Science,Literature,Fantasy,Historical,Fiction
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,1
2,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
734,0,0,1,0,0,0,0,0,1
735,0,0,1,0,0,0,1,0,1
736,0,0,0,0,0,0,0,0,1
737,0,0,1,0,0,0,1,0,1
